In [ ]:
import numpy as np
import sys
from multiprocessing import Queue, Process
import matplotlib
#matplotlib.use('Agg') #for using this notebook as script
from IPython import display as jupyter

import matplotlib.pyplot as plt

from definitions_direct import *

import data_utils
data_utils.init("definitions_direct")

import model_utils
model_utils.init("definitions_direct")

In [ ]:
# define and compile models
pre_generator, generator = model_utils.generate_and_compile_only_generator_models()

In [ ]:
# populate data souce
discrete_images_queue = Queue(10)
tasks = data_utils.populate_pre_train_queues(discrete_images_queue)

In [ ]:
# load pre trained weights
generator.load_weights("d:/downloads/models/directV3/fields/generator.model")

In [ ]:
running_train_loss = 0
train_history = [100]
decay = 0.9    
for iteration in range(10000*20):
    batch_L, batch_a, batch_b = discrete_images_queue.get()
    losses = pre_generator.train_on_batch(batch_L, [batch_a, batch_b])
    running_train_loss = (decay * running_train_loss + losses[0] + losses[1]) / (1 + decay)
    train_history.append(losses)

    if iteration % 25 == 0:
        data_utils.save_discrete_images(discrete_images_queue, pre_generator)
        pre_generator.save("pre_generator.model")
        jupyter.clear_output(wait=True)
    
    print("Averaged training loss=%f, Iteration=%d" % (running_train_loss, iteration))

In [ ]:
pre_generator.save("pre_generator.model")